In [2]:
import fastf1
import pandas as pd
import datetime
from unidecode import unidecode
from fastf1.logger import set_log_level

from fastf1.ergast import Ergast

In [3]:
set_log_level("DEBUG")
ergast = Ergast()

In [4]:
driverMapping = {}
raceMapping = {}
roundMapping = {}

master_weather_df = pd.DataFrame()
master_laps_df = pd.DataFrame()

In [5]:
def checkpoint():
    master_weather_df.to_csv('../output_csv/weather.csv', index=False)
    master_laps_df.to_csv('../output_csv/laps.csv', index=False)
    
def reload():
    global driverMapping
    global raceMapping
    global roundMapping
    global master_weather_df
    global master_laps_df
    
    driver_df = pd.read_csv('../output_csv/drivers.csv')
    race_df = pd.read_csv('../output_csv/race.csv')
    master_laps_df = pd.read_csv('../output_csv/laps.csv')
    master_weather_df = pd.read_csv('../output_csv/weather.csv')
    driver_data = driver_df.to_dict(orient='records')
    race_data = race_df.to_dict(orient='records')
    
    driverMapping = {}
    for driver in driver_data:
        full_name = driver['firstName'] + " " + driver['lastName']
        did = driver['dID']
        driverMapping[unidecode(full_name).lower()] = did
        
    raceMapping = {}
    roundMapping = {}
    for race in race_data:
        round = race['round']
        season = race['season']
        rid = race['cID']
        raceMapping[str(round) + "." + str(season)] = rid
        if season in roundMapping:
            roundMapping[season] += 1
        else:
            roundMapping[season] = 1

In [6]:
def querier(season):
    global master_weather_df
    global master_laps_df
    
    rounds = roundMapping[season]
    for round in range(1, rounds + 1):
        session = fastf1.get_session(season, round, 'R')
        session.load(weather=True)
        weather_df = session.weather_data
        
        def getDID(code):
            driver = session.get_driver(code)
            return driverMapping[unidecode(driver['FullName']).lower()]
        
        rid = raceMapping[str(round) + "." + str(season)]
        
        # weather
        weather_df['rID'] = rid
        weather_df = weather_df.drop(columns=['Pressure', 'Humidity', 'WindDirection'])
        weather_df['Time'] = weather_df['Time'].dt.total_seconds()
        weather_df = weather_df.iloc[::len(weather_df)//4]
        master_weather_df = pd.concat([master_weather_df, weather_df], ignore_index=True)
        
        # laps
        laps_df = session.laps[['Driver', 'Time', 'LapNumber', 'LapTime', 'Position', 'PitInTime', 'PitOutTime']]
        max_laps =laps_df['LapNumber'].max()
        
        laps_df['dID'] = laps_df['Driver'].apply(getDID)
        laps_df['rID'] = rid
        laps_df['Speed'] = laps_df['LapTime'].dt.total_seconds()
        laps_df['Time'] = laps_df['Time'].dt.total_seconds()
        laps_df['PitInTime'] = laps_df['PitInTime'].dt.total_seconds()
        laps_df['PitOutTime'] = laps_df['PitOutTime'].dt.total_seconds()
    
        laps_df = laps_df[
            (laps_df['LapNumber'] == 1) |                    # First lap
            (laps_df['LapNumber'] == max_laps) |              # Last lap
            (laps_df['PitInTime'].notna()) |                 # Has pit in time
            (laps_df['PitOutTime'].notna())
        ]
    
        laps_df = laps_df[
            laps_df['LapTime'].notna() &                      # Speed not null
            laps_df['Position'].notna()                     # Position not null
        ]
    
        master_laps_df = pd.concat([master_laps_df, laps_df], ignore_index=True)

In [79]:
reload()
querier(2018)
checkpoint()

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
_api          DEBUG 	Falling back to livetiming mirror (https://livetiming-mirror.fastf1.dev)
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req     

In [ ]:
reload()
querier(2019)
checkpoint()

In [83]:
reload()
querier(2020)
checkpoint()

core           INFO 	Loading data for Austrian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver  8: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '16', '4', '44', '55', '11', '10', '31', '99', '5', '6', '26', '23', '7', '63', '8', '20', '

In [95]:
reload()
querier(2021)
checkpoint()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '11'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 44 completed the race distance 00:00.067000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 

In [100]:
reload()
querier(2022)
checkpoint()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 16 completed the race distance 00:00.050000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', 

In [7]:
reload()
querier(2023)
checkpoint()

req         WARNING 	DEFAULT CACHE ENABLED! (10.16 GB) /Users/ruichen/Library/Caches/fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27'

In [7]:
reload()
querier(2024)
checkpoint()

req         WARNING 	DEFAULT CACHE ENABLED! (14.06 GB) /Users/ruichen/Library/Caches/fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23',